In [1]:
import tflearn
from tflearn.datasets import cifar100
import numpy as np

In [2]:
# Residual blocks
# 32 layers: n=5, 56 layers: n=9, 110 layers: n=18
n = 5

# Data loading

(X, Y), (testX, testY) = cifar100.load_data('./cifar-100-python')
Y = tflearn.data_utils.to_categorical(Y,nb_classes=100)
testY = tflearn.data_utils.to_categorical(testY,nb_classes=100)
X=X.astype(np.float32)
Y=Y.astype(np.float32)
testX=testX.astype(np.float32)
testY=testY.astype(np.float32)
# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
img_prep.add_featurewise_zero_center(per_channel=True)

# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_crop([32, 32], padding=4)

In [9]:
help(tflearn.conv_2d)

Help on function conv_2d in module tflearn.layers.conv:

conv_2d(incoming, nb_filter, filter_size, strides=1, padding='same', activation='linear', bias=True, weights_init='uniform_scaling', bias_init='zeros', regularizer=None, weight_decay=0.001, trainable=True, restore=True, reuse=False, scope=None, name='Conv2D')
    Convolution 2D.
    
    Input:
        4-D Tensor [batch, height, width, in_channels].
    
    Output:
        4-D Tensor [batch, new height, new width, nb_filter].
    
    Arguments:
        incoming: `Tensor`. Incoming 4-D Tensor.
        nb_filter: `int`. The number of convolutional filters.
        filter_size: `int` or `list of int`. Size of filters.
        strides: 'int` or list of `int`. Strides of conv operation.
            Default: [1 1 1 1].
        padding: `str` from `"same", "valid"`. Padding algo to use.
            Default: 'same'.
        activation: `str` (name) or `function` (returning a `Tensor`) or None.
            Activation applied to this lay

In [3]:
# Building Residual Network
tflearn.config.init_graph(gpu_memory_fraction=0.4)
net = tflearn.input_data(shape=[None, 32, 32, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
net = tflearn.residual_block(net, n, 16)
net = tflearn.residual_block(net, 1, 32, downsample=True)
net = tflearn.residual_block(net, n-1, 32)
net = tflearn.residual_block(net, 1, 64, downsample=True)
net = tflearn.residual_block(net, n-1, 64)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)
# Regression
net = tflearn.fully_connected(net, 100, activation='softmax')
mom = tflearn.Momentum(1., lr_decay=0.1, decay_step=32000, staircase=True)
net = tflearn.regression(net, optimizer=mom,
                         loss='categorical_crossentropy')
# Training
model = tflearn.DNN(net, checkpoint_path='model_resnet_cifar100',
                    max_checkpoints=10, tensorboard_verbose=0,
                    clip_gradients=0.)


In [12]:
help(tflearn.fully_connected)

Help on function fully_connected in module tflearn.layers.core:

fully_connected(incoming, n_units, activation='linear', bias=True, weights_init='truncated_normal', bias_init='zeros', regularizer=None, weight_decay=0.001, trainable=True, restore=True, reuse=False, scope=None, name='FullyConnected')
    Fully Connected.
    
    A fully connected layer.
    
    Input:
        (2+)-D Tensor [samples, input dim]. If not 2D, input will be flatten.
    
    Output:
        2D Tensor [samples, n_units].
    
    Arguments:
        incoming: `Tensor`. Incoming (2+)D Tensor.
        n_units: `int`, number of units for this layer.
        activation: `str` (name) or `function` (returning a `Tensor`).
            Activation applied to this layer (see tflearn.activations).
            Default: 'linear'.
        bias: `bool`. If True, a bias is used.
        weights_init: `str` (name) or `Tensor`. Weights initialization.
            (see tflearn.initializations) Default: 'truncated_normal'.
     

In [4]:
model.fit(X[::], Y[::], n_epoch=100, validation_set=(testX, testY),
          snapshot_epoch=False, snapshot_step=100,
          show_metric=True, batch_size=128, shuffle=True,
          run_id='resnet_cifar100')

Training Step: 37540  | total loss: 0.85350 | time: 0.362s
| Momentum | epoch: 097 | loss: 0.85350 - acc: 0.7365 -- iter: 00512/50000


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

